Next steps

~~target point cloud made from CAD file~~

fix filter/output on RANSAC 
+ ~~disable mutual filtering~~
+ change feature extraction params
+ less downsampling (can try to greater extent)
+ change feature types (SHOT or within FPFH)
+ change dist threshold
+ try fast global registration?



In [2]:
# READ DATA FILE

import open3d as o3d
import numpy as np
import os
import sys

pcd = o3d.io.read_point_cloud("JUST-TM-62.pcd")
print(pcd)
print(np.asarray(pcd.points))
# o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
PointCloud with 13939 points.
[[-0.275 -0.213 -2.349]
 [-0.241 -0.213 -2.347]
 [-0.174 -0.213 -2.346]
 ...
 [-0.8   -0.81  -2.306]
 [-0.75  -0.801 -2.309]
 [-0.704 -0.797 -2.322]]


In [110]:
# ALTERNATIVELY (PREFERRED) USE TM62 CAD FILE AS TARGET

mesh = o3d.io.read_triangle_mesh("Hollow_62_Clinic.stl")
point_cloud = mesh.sample_points_uniformly(number_of_points=100) # very sparse sampling, default is 100x higher
o3d.io.write_point_cloud("TM62_target.pcd", point_cloud)



True

In [102]:
# ignore this for now

def radius_search():
    print("Loading pointcloud ...")
    sample_pcd_data = o3d.data.PCDPointCloud()
    # pcd = o3d.io.read_point_cloud("p213.pcd")
    pcd = o3d.io.read_point_cloud("JUST-TM-62.pcd")
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    print(
        "Find the neighbors of 50000th point with distance less than 0.2, and painting them green ..."
    )
    [k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[50000], 0.2)
    np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]

    print("Displaying the final point cloud ...\n")
    o3d.visualization.draw([pcd])

def knn_search():
    print("Loading pointcloud ...")
    sample_pcd = o3d.data.PCDPointCloud()
    pcd = o3d.io.read_point_cloud(sample_pcd.path)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    print(
        "Find the 2000 nearest neighbors of 50000th point, and painting them red ..."
    )
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[50000], 2000)
    np.asarray(pcd.colors)[idx[1:], :] = [1, 0, 0]

    print("Displaying the final point cloud ...\n")
    o3d.visualization.draw([pcd])

# if __name__ == "__main__":
#     knn_search()
#     radius_search()

In [111]:
# DATA PREPROCESSING

# Load point clouds
source = o3d.io.read_point_cloud("pointCloud1.pcd", format='pcd')
target = o3d.io.read_point_cloud("TM62_target.pcd", format='pcd')

# want to scale down target
# Define the scaling factor (e.g., 0.5 to reduce size by half)
scaling_factor = 0.05

# Extract point coordinates
points = np.asarray(target.points)

# Apply scaling
scaled_points = points * scaling_factor

# Create a new point cloud with scaled points
scaled_pcd = o3d.geometry.PointCloud()
scaled_pcd.points = o3d.utility.Vector3dVector(scaled_points)

target = scaled_pcd

# Downsample and estimate normals
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    return pcd_down

def preprocess_point_cloud_wout_downsample(pcd, voxel_size):
    # pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    pcd_down = pcd
    return pcd_down

voxel_size = 0.05
source_down = preprocess_point_cloud(source, voxel_size) # TRY SKIPPING DOWNSAMPLING OF FIELD
target_down = preprocess_point_cloud(target, voxel_size)

# Apply manual scaling if needed
scale_factor = 1.0  # Adjust this value as needed
source_scaled = o3d.geometry.PointCloud()
source_scaled.points = o3d.utility.Vector3dVector(np.array(source_down.points) * scale_factor)
source_scaled.normals = source_down.normals  # Preserve normals



# # CROP TARGET POINT CLOUD TO JUST LANDMINE (not needed w/ CAD)

# # Define bounding box
# min_bound = np.array([-3, -3, -3])  # Z, X/Y
# max_bound = np.array([0, 0, -1])     

# # Create AxisAlignedBoundingBox
# box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

# # Crop point cloud
# cropped_target_pcd = target_down.crop(box)



# target_down = cropped_target_pcd

In [112]:
#o3d.visualization.draw_geometries([source_down])

o3d.visualization.draw_geometries([target_down])

In [113]:
# FPFH 

# Compute FPFH features
def compute_fpfh_features(pcd, voxel_size):
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*100, max_nn=500))
    return fpfh

source_fpfh = compute_fpfh_features(source_scaled, voxel_size)
target_fpfh = compute_fpfh_features(target_down, voxel_size)

# source_fpfh
# target_fpfh

In [114]:

# RANSAC

# RANSAC based alignment
distance_threshold = voxel_size * 10
mutual_filter = False
result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_scaled, target_down, source_fpfh, target_fpfh, mutual_filter, distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    4, [
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.999)
)

print(result_ransac.transformation)

# # Refine the alignment using ICP
# result_icp = o3d.pipelines.registration.registration_icp(
#     source_scaled, target_down, distance_threshold, result_ransac.transformation,
#     o3d.pipelines.registration.TransformationEstimationPointToPoint()
# )

# print(result_icp.transformation)


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [116]:
# VISUALIZE RESULTS

# Apply the RANSAC transformation to the source point cloud
source_transformed = source_scaled.transform(result_ransac.transformation)

# Assign different colors to the point clouds for better visualization
source_transformed.paint_uniform_color([1.0, 0.0, 0.0])  # Red
target_down.paint_uniform_color([0.0, 1.0, 0.0])  # Green

# Visualize the transformed source and target point clouds
o3d.visualization.draw_geometries([source_transformed, target_down],
                                   window_name="Transformed Source and Target Point Clouds",
                                   width=800, height=600)
